In [1]:
%matplotlib


Using matplotlib backend: TkAgg


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.dates as dates
import matplotlib.pyplot as plt

In [5]:
print os.getcwd()

/usr/local/google/home/soltesz/src/github.com/stephen-soltesz/ipython-testing


In [6]:
# generate data with:
# cat daily_discards.sql | bq query --format=csv --max_rows=1000000 --nouse_legacy_sql > daily_discards.csv
df = pd.read_csv('daily_discards.csv')

In [7]:
len(df)

86688

In [8]:
sites = [
    ['dfw', 'lga', 'iad'],
    ['sea', 'atl', 'den'],
    ['mia', 'nuq', 'ord'],
]

fig, axes = plt.subplots(nrows=3, ncols=3)
for i, site_row in enumerate(sites):
    for j, site in enumerate(site_row):
        axes[i, j].set_title(site)
        if j != 0:
            axes[i, j].set_yticklabels([])
        if i != len(sites)-1:
            axes[i, j].set_xticklabels([])
        for h in set(df['hostname']):
            if 'mlab1.' + site in h:
                ds = df[ (df['hostname'] == h) ]
                axes[i, j].plot_date(dates.epoch2num(ds['ts']), ds['total_discards'], ls='-', ms=0, label=h[:11])
        axes[i, j].set_ylim(1, 1000000)
        axes[i, j].tick_params(axis='x', labelrotation=90)
        axes[i, j].grid(color='#dddddd')
        axes[i, j].legend(loc=4, fontsize='x-small')
        axes[i, j].semilogy()
        
fig.subplots_adjust(hspace=0.3, wspace=0.4)
fig.suptitle('Discards over time')


Text(0.5,0.98,'Discards over time')

In [9]:
len(df['hostname'])

86688

In [11]:
df = pd.read_csv('total_discards.csv')

In [12]:
sites = [
    ['dfw', 'lga', 'iad'],
    ['sea', 'atl', 'den'],
    ['mia', 'nuq', 'ord'],
]
#sites = [
#    ['dfw02']
#]

fig, axes = plt.subplots(nrows=3, ncols=3)
#fig, axes = plt.subplots(nrows=2, ncols=2)
for i, site_row in enumerate(sites):
    for j, site in enumerate(site_row):
        axes[i, j].set_title(site)
        if j != 0:
            axes[i, j].set_yticklabels([])
        if i != len(sites)-1:
            axes[i, j].set_xticklabels([])
        for h in set(df['hostname']):
            if 'mlab1.' + site in h:
                # -- ds_discards = df[ (df['hostname'] == h) & (df['metric'] == 'discards') ]
                # ds_total = df[ (df['hostname'] == h) & (df['metric'] == 'octets') ]
                ds = df['total_discards'][ df['hostname'] == h ] / df['total_packets'][df['hostname'] == h]
                axes[i, j].plot_date(dates.epoch2num(df['ts'][df['hostname'] == h]), ds, ls='-', ms=0, label=h[:11])
        axes[i, j].set_ylim(10**-6, 10**-3)
        axes[i, j].tick_params(axis='x', labelrotation=90)
#       axes[i, j].set_xticks(rotation='vertical')
        axes[i, j].grid(color='#dddddd')
        axes[i, j].legend(loc=4, fontsize='x-small')
        axes[i, j].semilogy()
        
fig.subplots_adjust(hspace=0.3, wspace=0.4)
fig.suptitle('Ratio of discard / total packets')

Text(0.5,0.98,'Ratio of discard / total packets')

/usr/local/google/home/soltesz/.local/lib/python2.7/site-packages/matplotlib/scale.py:114: RuntimeWarning: invalid value encountered in less_equal
  out[a <= 0] = -1000


In [15]:
# cat sidestream.sql | bq query --format=csv --max_rows=1000000 --nouse_legacy_sql > sidestream-trial-6w.csv
df = pd.read_csv('sidestream-trial-6w.csv')

In [16]:
print len(df), max(df['rate_mbps'])

hosts = [
    ['mlab1.lga03', 'mlab1.lga04'],
    ['mlab1.dfw02', 'mlab1.dfw03'],
]

fig, axes = plt.subplots(nrows=2, ncols=2)

for i, host_row in enumerate(hosts):
    for j, host in enumerate(host_row):
        for period in ['0w', '1w', '2w']: # , '3w', '4w', '5w']:
            ds = df[ (df['period'] == period) & (df['hostname'] == host) ]
            n, bins, patches = axes[i, j].hist(ds['rate_mbps'], len(ds['rate_mbps']),
                               histtype='step', normed=1, cumulative=True, label='cdf-' + period,
                               ls='-')
        # axes[i].set_aspect('equal')
        axes[i, j].set_axisbelow(True)
        axes[i, j].semilogx()
        axes[i, j].legend(loc=2)
        axes[i, j].grid(color='#dddddd')
        axes[i, j].set_title(host)

fig.suptitle('Sidestream Download Rate CDFs over three week period (0w is trial)')
plt.show()
print n, len(bins)

262916 901.2757078134047


/usr/local/google/home/soltesz/.local/lib/python2.7/site-packages/matplotlib/axes/_axes.py:6448: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


[0.03767123 0.05950342 0.07719749 ... 0.99985731 0.99985731 1.        ] 7009


In [28]:
# cat daily_pct.sql| bq query --format=csv --max_rows=1000000 --nouse_legacy_sql > daily_pct.csv
df = pd.read_csv('daily_pct.csv')

In [36]:
#plt.plot_date(dates.epoch2num(df['ts']), df['total_discards'], ls='-', ms=0)
#plt.tick_params(axis='x', labelrotation=90)
#plt.title('dfw02 percent of daily timebins with any discards')
#plt.show()

sites = [
    ['mlab1.dfw02', 'mlab1.lga03'],
]

fig, axes = plt.subplots(nrows=1, ncols=2)
for i, hosts in enumerate(sites):
    for j, host in enumerate(hosts): 
        ax = axes[j]
        
        ds = df[ df['hostname'] == host ]
        ax.plot_date(dates.epoch2num(ds['ts']), ds['total_discards'], ls='-', ms=0, label=host)
        
        ax.set_title(host)
        #ax.set_ylim(10**-6, 10**-3)
        ax.tick_params(axis='x', labelrotation=90)
        ax.grid(color='#dddddd')
        ax.legend(loc=4, fontsize='x-small')
        #ax.semilogy()
        
fig.subplots_adjust(hspace=0.3, wspace=0.4)
fig.suptitle('Daily percentage of timebins with any discards')

Text(0.5,0.98,'Daily percentage of timebins with any discards')

In [42]:
# cat sidestream-aug1st.sql | bq query --format=csv --max_rows=1000000 --nouse_legacy_sql > sidestream-aug.csv
df = pd.read_csv('sidestream-aug.csv')

In [44]:
print len(df), max(df['rate_mbps'])

hosts = [
    ['mlab1.dfw02', 'mlab1.lga03'],
]

fig, axes = plt.subplots(nrows=1, ncols=2)

for i, host_row in enumerate(hosts):
    for j, host in enumerate(host_row):
        for period in set(df['period']):
            ds = df[ (df['period'] == period) & (df['hostname'] == host) ]
            ax = axes[j]
            n, bins, patches = ax.hist(ds['rate_mbps'], len(ds['rate_mbps']),
                               histtype='step', normed=1, cumulative=True, label='cdf-' + period,
                               ls='-')
        # axes[i].set_aspect('equal')
        ax.set_xlim(1, 100)

        ax.set_axisbelow(True)
        #ax.semilogx()
        ax.legend(loc=2)
        ax.grid(color='#dddddd')
        ax.set_title(host)

fig.suptitle('Sidestream Download Rate CDFs Around August 5th')

plt.show()
print n, len(bins)

240746 880.9183945638188
[0.00793568 0.03320455 0.04114023 ... 0.99989558 0.99989558 1.        ] 9578
